In [130]:
import pyspark
import numpy as np
import pandas as pd
from pyspark.sql import SparkSession

pd.set_option("max_colwidth", 800)

In [131]:
# Run Spark on localhost
spark = SparkSession\
    .Builder()\
    .config("spark.driver.host", "127.0.0.1")\
    .appName("wiki_bias")\
    .getOrCreate()

In [132]:
import os
for dirname, _, filenames in os.walk("/Users/chriswallerstein/Development/python/wikipedia_bias/data"):
    for file in filenames:
        print(os.path.join(dirname, file))

wiki_data = spark.read\
    .option("mode", "dropmalformed")\
    .option("inferSchema", "true")\
    .option("header", "true")\
    .option("multiline", "true")\
    .option("charset", "UTF-8")\
    .csv("/Users/chriswallerstein/Development/python/wikipedia_bias/data/*.csv")

/Users/chriswallerstein/Development/python/wikipedia_bias/data/wiki_corpus_2022_2_13.csv


In [133]:
# basic data cleansing
import pyspark.sql.functions as F

@F.udf
def ascii_ignore(x):
    return x.encode("ascii", "ignore").decode("ascii")

wiki_data = wiki_data.dropna()
wiki_data = wiki_data.withColumn("sentence_text", ascii_ignore("sentence"))

In [134]:
# tokenize
from pyspark.ml.feature import RegexTokenizer

regex_tokenizer = RegexTokenizer(inputCol="sentence_text", outputCol="words", pattern="\\W")
wiki_data = regex_tokenizer.transform(wiki_data)
wiki_data.sample(True, 0.05).toPandas()

,id,source,sentence,sentence_text,words
0,20220213123311980643,rational,"in particular, hip hop/country/southern rock/heartland rock artist kid rock (who is actually from detroit, like his good friend ted nugent, but recently moved to alabama and has gone on record as a raging libertarian or conservative, depending on the issue) has shouted down protests from his hometown to stop and/or apologize.http://www.rollingstone.com/music/news/kid-rock-tells-protestors-to-kiss-my-ass-over-confederate-flag-20150710=====2015=====in the wake of the charleston shootings, the battle flag came to increasing prominence due to pictures of an idiot with a bowl cut the confessed shooter with the flag.","in particular, hip hop/country/southern rock/heartland rock artist kid rock (who is actually from detroit, like his good friend ted nugent, but recently moved to alabama and has gone on record as a raging libertarian or conservative, depending on the issue) has shouted down protests from his hometown to stop and/or apologize.http://www.rollingstone.com/music/news/kid-rock-tells-protestors-to-kiss-my-ass-over-confederate-flag-20150710=====2015=====in the wake of the charleston shootings, the battle flag came to increasing prominence due to pictures of an idiot with a bowl cut the confessed shooter with the flag.","[in, particular, hip, hop, country, southern, rock, heartland, rock, artist, kid, rock, who, is, actually, from, detroit, like, his, good, friend, ted, nugent, but, recently, moved, to, alabama, and, has, gone, on, record, as, a, raging, libertarian, or, conservative, depending, on, the, issue, has, shouted, down, protests, from, his, hometown, to, stop, and, or, apologize, http, www, rollingstone, com, music, news, kid, rock, tells, protestors, to, kiss, my, ass, over, confederate, flag, 20150710, 2015, in, the, wake, of, the, charleston, shootings, the, battle, flag, came, to, increasing, prominence, due, to, pictures, of, an, idiot, with, a, bowl, cut, the, confessed, ...]"
1,20220213123311980773,rational,"neo-nazis in germany fly the confederate flag because it is illegal to display nazi symbols.the russian invasions of ukraine in 2014 were supported by american white supremacists richard spencer, matthew heimbach, and david duke.","neo-nazis in germany fly the confederate flag because it is illegal to display nazi symbols.the russian invasions of ukraine in 2014 were supported by american white supremacists richard spencer, matthew heimbach, and david duke.","[neo, nazis, in, germany, fly, the, confederate, flag, because, it, is, illegal, to, display, nazi, symbols, the, russian, invasions, of, ukraine, in, 2014, were, supported, by, american, white, supremacists, richard, spencer, matthew, heimbach, and, david, duke]"
2,20220213123311980865,rational,"does it remind you of islam?flag_of_alabama_(1861,_reverse).svg|the alabama flag (reverse side).","does it remind you of islam?flag_of_alabama_(1861,_reverse).svg|the alabama flag (reverse side).","[does, it, remind, you, of, islam, flag_of_alabama_, 1861, _reverse, svg, the, alabama, flag, reverse, side]"
3,20220213123312203753,rational,"la orientación sexual es la dirección que toma la sexualidad del individuo, usualmente clasificada en relación al sexo o género por el que se siente atraído.","la orientacin sexual es la direccin que toma la sexualidad del individuo, usualmente clasificada en relacin al sexo o gnero por el que se siente atrado.","[la, orientacin, sexual, es, la, direccin, que, toma, la, sexualidad, del, individuo, usualmente, clasificada, en, relacin, al, sexo, o, gnero, por, el, que, se, siente, atrado]"
4,20220213123312358984,rational,"a single photon of frequency 5.4×1014 hz, the frequency of visible yellow light, has a paltry 3.6×10-19 joules of energy.","a single photon of frequency 5.41014 hz, the frequency of visible yellow light, has a paltry 3.610-19 joules of energy.","[a, single, photon, of, frequency, 5, 41014, hz, the, frequency, of, visible, yellow, ligh

In [135]:
# vectorize words
from pyspark.ml.feature import CountVectorizer

cv = CountVectorizer(inputCol="words", outputCol="features")
cv_model = cv.fit(wiki_data)
wiki_data = cv_model.transform(wiki_data)
wiki_data.sample(True, 0.05).toPandas()

,id,source,sentence,sentence_text,words,features
0,20220213123309464889,conservapedia,"""feisal abdul rauf is a muslim religious leader, activist and author who is spearheading the proposal to construct an islamic cultural center two blocks from the wreckage of the world trade center.imam feisal abdul rauf's goodwill tour comes amid 'ground zero mosque' furor the media have dubbed the project the """"ground zero mosque"""" from its nearness to ground zero and because it will include a mosque.""\r","""feisal abdul rauf is a muslim religious leader, activist and author who is spearheading the proposal to construct an islamic cultural center two blocks from the wreckage of the world trade center.imam feisal abdul rauf's goodwill tour comes amid 'ground zero mosque' furor the media have dubbed the project the """"ground zero mosque"""" from its nearness to ground zero and because it will include a mosque.""\r","[feisal, abdul, rauf, is, a, muslim, religious, leader, activist, and, author, who, is, spearheading, the, proposal, to, construct, an, islamic, cultural, center, two, blocks, from, the, wreckage, of, the, world, trade, center, imam, feisal, abdul, rauf, s, goodwill, tour, comes, amid, ground, zero, mosque, furor, the, media, have, dubbed, the, project, the, ground, zero, mosque, from, its, nearness, to, ground, zero, and, because, it, will, include, a, mosque]","(6.0, 1.0, 2.0, 2.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 1.0, 0.0, 2.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...)"
1,20220213123311786617,rational,"david koch was responsible for founding americans for prosperity, a teabagger astroturf group.","david koch was responsible for founding americans for prosperity, a teabagger astroturf group.","[david, koch, was, responsible, for, founding, americans, for, prosperity, a, teabagger, astroturf, group]","(0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...)"
2,20220213123311786627,rational,koch denied involvement but video later leaked of him at teabagger parties.,koch denied involvement but video later leaked of him at teabagger parties.,"[koch, denied, involvement, but, video, later, leaked, of, him, at, teabagger, parties]","(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...)"
3,20220213123311980156,rational,"thumb|right|165px|the battle flag, here shown with the proper aspect ratio.","thumb|right|165px|the battle flag, here shown with the proper aspect ratio.","[thumb, right, 165px, the, battle, flag, here, shown, with, the, proper, aspect, ratio]","(2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [136]:
# transform label column (source)
from pyspark.ml.feature import StringIndexer

si = StringIndexer(inputCol="source", outputCol="label")
si_model = si.fit(wiki_data)
wiki_data = si_model.transform(wiki_data)
wiki_data.sample(True, 0.05).toPandas()

,id,source,sentence,sentence_text,words,features,label
0,20220213123311273317,conservapedia,"the breadth and variety of his activities and associations, and his ability to describe them vividly in writing, earned him international fame as lawrence of arabia, a title used for the 1962 film based on his wartime activities.","the breadth and variety of his activities and associations, and his ability to describe them vividly in writing, earned him international fame as lawrence of arabia, a title used for the 1962 film based on his wartime activities.","[the, breadth, and, variety, of, his, activities, and, associations, and, his, ability, to, describe, them, vividly, in, writing, earned, him, international, fame, as, lawrence, of, arabia, a, title, used, for, the, 1962, film, based, on, his, wartime, activities]","(2.0, 2.0, 3.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...)",2.0
1,20220213123311786627,rational,koch denied involvement but video later leaked of him at teabagger parties.,koch denied involvement but video later leaked of him at teabagger parties.,"[koch, denied, involvement, but, video, later, leaked, of, him, at, teabagger, parties]","(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...)",0.0
2,20220213123311786731,rational,snyder signed it into law despite referendums telling him not to.,snyder signed it into law despite referendums telling him not to.,"[snyder, signed, it, into, law, despite, referendums, telling, him, not, to]","(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...)",0.0
3,20220213123311980336,rational,"unfortunately, it was also adopted as a symbol emblazoned on the white sheets of a bunch of idiots shortly thereafter,one theory ties this to a desire to appear as ghosts of the war dead.","unfortunately, it was also adopted as a symbol emblazoned on the white sheets of a bunch of idiots shortly thereafter,one theory ties this to a desire to appear as ghosts of the war dead.","[unfortunately, it, was, also, adopted, as, a, symbol, emblazoned, on, the, white, sheets, of, a, bunch, of, idiots, shortly, thereafter, one, theory, ties, this, to, a, desire, to, appear, as, ghosts, of, the, war, dead]","(2.0, 3.0, 0.0, 2.0, 0.0, 3.0, 1.0, 2.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0

In [137]:
train, test = wiki_data.select("features", "label").randomSplit([0.9,0.1])
print(train.count())
print(test.count())

539
56


In [138]:
# Try Logistic regression
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression()
lr_model = lr.fit(train)
lr_predictions = lr_model.transform(test)
lr_predictions.limit(5).toPandas()

,features,label,rawPrediction,probability,prediction
0,"(3.0, 1.0, 2.0, 2.0, 1.0, 1.0, 0.0, 2.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...)",0.0,"[12.110304106810734, -3.206377919342339, -8.903926187468395]","[0.9999997763835844, 2.2286887347190357e-07, 7.475420800285711e-10]",0.0
1,"(5.0, 1.0, 1.0, 1.0, 3.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 3.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...)",0.0,"[31.617782787282025, -16.639744971806103, -14.978037815475918]","[1.0, 1.1015950707415723e-21, 5.8035299826442485e-21]",0.0
2,"(2.0, 1.0, 2.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 2.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...)",0.0,"[13.72602006475191, -2.2450515512500564, -11.480968513501853]","[0.9999998841505419, 1.1583816701534736e-07, 1.1291298917678416e-11]",0.0
3,"(9.0, 3.0, 4.0, 3.0, 2.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...)",0.0,"[9.896213821826361, -2.0296672210330216, -7.866546600793347]","[0.9999933638195524, 6.616872848106036e-06, 1.930759956252672e-08]",0.0
4,"(3.0, 4.0, 1.0, 2.0, 2.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...)",0.0,"[14.756575644500764, -5.962374679345094, -8.794200965155671]","[0.9999999989365151, 1.0043248372686986e-09, 5.915993299341227e-11]",0.0


In [139]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator()
print(f"Linear regression f1 {evaluator.evaluate(lr_predictions):.4f}")

Linear regression f1 0.9031


In [140]:
training_summary = lr_model.summary

